In [1]:
import shutil
from deepCR import deepCR
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import cm
import h5py
import numpy as np
import os
import gc
import glob as glob
import shutil
from astropy.io import ascii
from astropy.table import Table
from astropy.visualization import ZScaleInterval
from astroquery.mast import Observations
from ccdproc import ImageFileCollection
from drizzlepac import tweakreg
from drizzlepac import astrodrizzle
from IPython.display import Image
%matplotlib inline

The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


## F435W

## F606W

## F814W

## Download and AstroDrizzle

In [4]:
f435w_obs_ids = ['j9es02010', 'j9es07010', 'j6lp01010', #Globular cluster, Globular cluster, Globular cluster
                 'j6lp03010', 'j6lp05010', 'j9ju02010', #Globular cluster, Globular cluster, Resolved galaxy (M31)
                 'j9ju04010', 'j9df03010', 'j9df01010', #Resolved galaxy (M31), Extragalactic field, Extragalactic field,
                 'j9df04010', 'jbfla3010', 'jc8mb1010', #Extragalactic field, Extragalactic field, Extragalactic field
                 'jcoy02030', 'jcoy03020', 'j8d6a3010'] #Resolved galaxy' (M101), Resolved galaxy' (M101), Resolved galaxy' (M101)
# jcoy02030
f606w_obs_ids = ['jcxx01010', 'jcxx06010', 'jc3f01010', #Extragalactic field, Extragalactic field, Extragalactic field
                 'j8z007010', 'j8z005010', 'j96q03010', #Resolved galaxy (M31) - Same proposal ID but different n_vis, Resolved galaxy (M31) - Same proposal but different n_vis, Resolved galaxy (M31)
                 'jc9704010', 'jc9707010', 'jcdm95020', #Extragalactic field, Extragalactic field, Resolved galaxy (M101)
                 'j90o28050', 'j90o13010', 'j90a04010', #Resolved galaxy (M33) - Same proposal but different n_vis, Resolved galaxy (M33) - Same proposal but different n_vis, Globular cluster,
                 'j90a03010', 'jcx618010', 'jb1605010'] #Globular cluster, Globular cluster, Globular cluster
# jb1605010
f814w_obs_ids = ['j8d6a2030', 'j8iy6d010', 'j8iy4b010', #Resolved galaxy (M101), Extragalactic field, Extragalactic field
                 'j8d414010', 'j8xi01010', 'jcnw11010', #Extragalactic field, Extragalactic field, Resolved galaxy (M32)
                 'jbf312010', 'jbf316010', 'jbf301010', #Resolved galaxy (M31), Resolved galaxy (M31), Resolved galaxy (M31)
                 'j8d416010', 'j9l952020', 'jbqj01030', #Extragalactic field, Globular cluster, Globular cluster
                 'jbqj02010', 'j9l929020', 'j9l9ad020'] #Globular cluster, Globular cluster, Globular cluster
# jbf301010

In [3]:
f435_prop_ids = [10595, 9442, 10760, 10631, 12103, 13498, 13737, 9490]
f606_prop_ids = [14343, 13057, 10260, 10407, 13375, 13364, 10190, 10146, 14235, 11586]
f814_prop_ids = [9490, 9405, 9450, 10092, 13691, 12058, 10775, 12602]

In [19]:
err_obs_ids = ['jcoy02030', 'jb1605010', 'jbf301010']

In [251]:
f435w_Table = Observations.query_criteria(obstype='all',
                                      obs_collection='HST',
                                      obs_id = f435w_obs_ids)
f606w_Table = Observations.query_criteria(obstype='all',
                                      obs_collection='HST',
                                      obs_id = f606w_obs_ids)
f814w_Table = Observations.query_criteria(obstype='all',
                                      obs_collection='HST',
                                      obs_id = f814w_obs_ids)

In [20]:
err_Table = Observations.query_criteria(obstype = 'all',
                                       obs_collection='HST',
                                       obs_id = err_obs_ids)

In [22]:
err_product = Observations.get_product_list(err_Table)
Observations.download_products(err_product,
                              download_dir = '/Users/jameskwon/Documents/Research/Data/deepcr/err',
                              mrp_only = False,
                              productSubGroupDescription='FLC')

Local Path,Status,Message,URL
str97,str8,object,object
/Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jb1605iuq/jb1605iuq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jb1605ivq/jb1605ivq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jb1605jcq/jb1605jcq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jb1605jeq/jb1605jeq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jb1605jgq/jb1605jgq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jbf301hjq/jbf301hjq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jbf301hmq/jbf301hmq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jbf301hpq/jbf301hpq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jbf301htq/jbf301htq_flc.fits,COMPLETE,None,None


In [253]:
# f435w_product = Observations.get_product_list(f435w_Table)
# Observations.download_products(f435w_product,
#                               download_dir = '/Users/jameskwon/Documents/Research/Data/deepcr/f435w',
#                               mrp_only = False,
#                               productSubGroupDescription='FLC')
f606w_product = Observations.get_product_list(f606w_Table)
Observations.download_products(f606w_product,
                              download_dir = '/Users/jameskwon/Documents/Research/Data/deepcr/f606w',
                              mrp_only = False,
                              productSubGroupDescription='FLC')
f814w_product = Observations.get_product_list(f814w_Table)
Observations.download_products(f814w_product,
                              download_dir = '/Users/jameskwon/Documents/Research/Data/deepcr/f814w',
                              mrp_only = False,
                              productSubGroupDescription='FLC')

Local Path,Status,Message,URL
str99,str8,object,object
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8d414vqq/j8d414vqq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8d414vsq/j8d414vsq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8d414vvq/j8d414vvq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8d414vyq/j8d414vyq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8d416vyq/j8d416vyq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8d416vzq/j8d416vzq_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8d416w1q/j8d416w1q_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8d416w3q/j8d416w3q_flc.fits,COMPLETE,None,None
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/mastDownload/HST/j8d6a2i8q/j8d6a2i8q_flc.fits,COMPLETE,None,None


In [9]:
base_dir = os.environ.get('DEEPCR_DIR')
f606_dir = os.path.join(base_dir,'f606w')
f435_dir = os.path.join(base_dir,'f435w')
f814_dir = os.path.join(base_dir,'f814w')

f606_mast_dir = os.path.join(f606_dir,'mastDownload','HST')
f435_mast_dir = os.path.join(f435_dir,'mastDownload','HST')
f814_mast_dir = os.path.join(f814_dir,'mastDownload','HST')

In [255]:
for prop_id in f606_prop_ids:
    dir_exist = os.path.isdir(base_dir+f'/f606w/{prop_id}')
    if not dir_exist:
        os.mkdir(base_dir+f'/f606w/{prop_id}')

In [256]:
f606_obs_table = Observations.query_criteria(obs_collection='HST', 
                                        filters='F606W', 
                                        instrument_name='ACS/WFC')

In [257]:
for prop_id in f435_prop_ids:
    dir_exist = os.path.isdir(base_dir+f'/f435w/{prop_id}')
    if not dir_exist:
        os.mkdir(base_dir+f'/f435w/{prop_id}')

In [258]:
f435_obs_table = Observations.query_criteria(obs_collection='HST', 
                                        filters='F435W', 
                                        instrument_name='ACS/WFC')

In [259]:
for prop_id in f814_prop_ids:
    dir_exist = os.path.isdir(base_dir+f'/f814w/{prop_id}')
    if not dir_exist:
        os.mkdir(base_dir+f'/f814w/{prop_id}')

In [260]:
f814_obs_table = Observations.query_criteria(obs_collection='HST', 
                                        filters='F814W', 
                                        instrument_name='ACS/WFC')

In [262]:
for f_download in os.listdir(f606_mast_dir):
    sub_dir = os.path.join(f606_mast_dir,f_download)
    if os.path.isdir(sub_dir):
        file = os.listdir(sub_dir)
        if len(file) != 1:
            print(sub_dir)
            print(file)
            raise ValueError
        else:

            file = file[0]
            print(file)
            file_dir = os.path.join(sub_dir,file)
            if os.path.isfile(file_dir):
                idx = (file.index('_flc.fits'))
                for i,ID in enumerate(f606_obs_table['obs_id']):
                    if file[:idx-5] in ID:
                        break
                prop_id = f606_obs_table['proposal_id'][i]
                src = file_dir
                print(os.path.join(f606_dir,prop_id))
                if os.path.isdir(os.path.join(f606_dir,prop_id)):
                    dst = os.path.join(f606_dir,prop_id,file)
                    shutil.move(src,dst)
                else:
                    raise ValueError

jcx618ilq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14235
jc3f01b8q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13057
jc9707rzq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13375
jcx618ioq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14235
j90o28fjq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10190
jcxx01bbq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343
jcxx01b0q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343
jc9704dxq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13375
jc9704drq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13375
j8z005nkq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260
j96q03i9q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10407
j8z005nhq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260
j90a04mtq_flc.fits
/Users/jameskwon/Documents/Resear

In [264]:
for f_download in os.listdir(f814_mast_dir):
    sub_dir = os.path.join(f814_mast_dir,f_download)
    if os.path.isdir(sub_dir):
        file = os.listdir(sub_dir)
        if len(file) != 1:
            print(sub_dir)
            print(file)
            raise ValueError
        else:

            file = file[0]
            print(file)
            file_dir = os.path.join(sub_dir,file)
            if os.path.isfile(file_dir):
                idx = (file.index('_flc.fits'))
                for i,ID in enumerate(f814_obs_table['obs_id']):
                    if file[:idx-5] in ID:
                        break
                prop_id = f814_obs_table['proposal_id'][i]
                src = file_dir
                print(os.path.join(f814_dir,prop_id))
                if os.path.isdir(os.path.join(f814_dir,prop_id)):
                    dst = os.path.join(f814_dir,prop_id,file)
                    shutil.move(src,dst)
                else:
                    raise ValueError

jbf301htq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058
j8d416w1q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/9450
j8xi01vxq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092
j9l952nlq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775
j9l9adanq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775
jbqj01ivq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602
j8d414vqq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/9450
jbf312f6q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058
j9l929ecq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775
j8iy4bduq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/9405
jbf316hnq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058
j8d416vyq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/9450
j8xi01w0q_flc.fits
/Users/jameskwon/Documents/Research/D

In [266]:
for f_download in os.listdir(f435_mast_dir):
    sub_dir = os.path.join(f435_mast_dir,f_download)
    if os.path.isdir(sub_dir):
        file = os.listdir(sub_dir)
        if len(file) != 1:
            print(sub_dir)
            print(file)
            raise ValueError
        else:

            file = file[0]
            print(file)
            file_dir = os.path.join(sub_dir,file)
            if os.path.isfile(file_dir):
                idx = (file.index('_flc.fits'))
                for i,ID in enumerate(f435_obs_table['obs_id']):
                    if file[:idx-5] in ID:
                        break
                prop_id = f435_obs_table['proposal_id'][i]
                src = file_dir
                print(os.path.join(f435_dir,prop_id))
                if os.path.isdir(os.path.join(f435_dir,prop_id)):
                    dst = os.path.join(f435_dir,prop_id,file)
                    shutil.move(src,dst)
                else:
                    raise ValueError

jcoy03ndq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/13737
j9ju04evq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760
j9ju02hyq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760
j9es07smq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595
j9df04idq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631
j9ju02i8q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760
j9ju04f1q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760
jcoy02crq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/13737
j9df01jqq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631
j6lp01egq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442
j9ju02i2q_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760
jcoy02dnq_flc.fits
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/13737
j9es02kvq_flc.fits
/Users/jameskwon/Documents/Researc

In [5]:
from drizzlepac import astrodrizzle

In [6]:
repo_dir = os.environ.get('DEEPCR')
repo_dir = os.path.join(repo_dir,'codes')

In [7]:
def remove_extra(data_dir):
    repo_dir = os.environ.get('DEEPCR')
    repo_dir = os.path.join(repo_dir,'codes')
    rm_list = []
    for file in os.listdir(repo_dir):
        if 'OrIg_' in file:
            pass
        elif 'final_' in file:
            rm_list.append(file)
        elif '_sci2.fits' in file:
            rm_list.append(file)
        elif '_sci1.fits' in file:
            rm_list.append(file)
        elif '_staticMask.fits' in file:
            rm_list.append(file)
        elif 'astrodrizzle.log' in file:
            rm_list.append(file)
    a = os.listdir(repo_dir)
    
    for f_rm in rm_list:
        rm_dir = os.path.join(repo_dir,f_rm)
        os.remove(rm_dir)
    
    rm_list = []
    for file in os.listdir(data_dir):
        if 'blt.fits' in file:
            rm_list.append(file)
        elif '_final_mask.fits' in file:
            rm_list.append(file)
        
        elif '_single_mask.fits' in file:
            rm_list.append(file)
        
        elif '_single_sci.fits' in file:
            rm_list.append(file)
        
        elif '_single_wht.fits' in file:
            rm_list.append(file)

    for f_rm in rm_list:
        rm_dir = os.path.join(data_dir,f_rm)
        os.remove(rm_dir)

In [11]:
for prop_id in os.listdir(f606_dir):
    sub_dir = os.path.join(f606_dir,prop_id)
    if os.path.isdir(sub_dir):
        if prop_id != 'mastDownload':
            for visit_num in os.listdir(sub_dir):
                if visit_num != '.DS_Store':
                    subsub_dir = os.path.join(sub_dir,visit_num)
                    print(subsub_dir)
                    drizzle_path = subsub_dir + '/*flc.fits'
                    try:
                        astrodrizzle.AstroDrizzle(drizzle_path)
                    except:
                        pass
                    remove_extra(subsub_dir)

/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/7
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 21:18:58.233 (29/06/2020)

==== Processing Step  Initialization  started at  21:18:58.236 (29/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 10.587296107210427  41.55185534661992  
CRPIX : 2108.5  2121.5  
CD1_1 CD1_2  : 1.388888888725236e-05  2.1321155967867122e-10  
CD2_1 CD2_2  : 2.1321155967867122e-10  -1.388888888725236e-05  
NAXIS : 4217  4243
********************************************************************************
*
*  Estimated memory usage:  up to 1612 Mb.
*  Output image size:       4217 X 4243 pixels. 
*  Output image file:       ~ 204 Mb. 
*  Cores available:         6
*
********************************************************************************
==== Processing Step Initialization finished at 21:18:59.497 (29/06/2020)
==== Processing Step  Static Mask  star

Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/7/j8z007nuq_sci2_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/7/j8z007nwq_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/7/j8z007nwq_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/7/j8z007nwq_sci1_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/7/j8z007nwq_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/7/j8z007nwq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/7/j8z007nwq_sci2_blt.fits
==== Processing Step Blot finished at 21:19:22.513 (29/06/2020)
==== Processi

      - EXT = 'SCI',2   delta(MDRIZSKY) = 644.434   NEW MDRIZSKY = 644.434
***** skymatch ended on 2020-06-29 21:19:47.200768
TOTAL RUN TIME: 0:00:02.156446
==== Processing Step Subtract Sky finished at 21:19:47.520 (29/06/2020)
==== Processing Step  Separate Drizzle  started at  21:19:47.521 (29/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 10.78915646240283  41.3736357334938  
CRPIX : 2108.5  2121.5  
CD1_1 CD1_2  : 1.3888888887259923e-05  2.1271832856845365e-10  
CD2_1 CD2_2  : 2.1271832856845365e-10  -1.3888888887259923e-05  
NAXIS : 4217  4243
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/5/j8z005nkq_single_sci.fits
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/5/j8z005niq_single_sci.fits
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/5/j8z005nhq_single_sci.fits
Writing out image to disk: /Users/jameskwon/

CD2_1 CD2_2  : -1.2351730273917451e-05  -6.351062415713797e-06  
NAXIS : 4219  4243
********************************************************************************
*
*  Estimated memory usage:  up to 1613 Mb.
*  Output image size:       4219 X 4243 pixels. 
*  Output image file:       ~ 204 Mb. 
*  Cores available:         6
*
********************************************************************************
==== Processing Step Initialization finished at 21:20:24.640 (29/06/2020)
==== Processing Step  Static Mask  started at  21:20:24.643 (29/06/2020)

==== Processing Step Static Mask finished at 21:20:25.894 (29/06/2020)
==== Processing Step  Subtract Sky  started at  21:20:25. (29/06/2020)

***** skymatch started on 2020-06-29 21:20:26.230244
      Version 1.0.5

'skymatch' task will apply computed sky differences to input image file(s).

NOTE: Computed sky values WILL NOT be subtracted from image data ('subtractsky'=False).
'MDRIZSKY' header keyword will represent sky value *compute

    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10407/3/j96q03idq_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10407/3/j96q03idq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10407/3/j96q03idq_sci2_blt.fits
==== Processing Step Blot finished at 21:20:46.301 (29/06/2020)
==== Processing Step  Driz_CR  started at  21:20:46.30 (29/06/2020)

Removed old cosmic ray mask file: '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10407/3/j96q03i9q_sci1_crmask.fits'
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10407/3/j96q03i9q_sci1_crmask.fits
Removed old cosmic ray mask file: '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10407/3/j96q03i6q_sci1_crmask.fits'
Removed old cosmic ray mask file: '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10407/3/j96q03idq

...

      - EXT = 'SCI',2   delta(MDRIZSKY) = 352.186   NEW MDRIZSKY = 352.186
   *   Image:   'j90a03gaq_flc.fits['SCI',1,2]'  --  SKY = 107.27483796353751 (brightness units)
       Sky change (data units):
      - EXT = 'SCI',1   delta(MDRIZSKY) = 337.648   NEW MDRIZSKY = 337.648
      - EXT = 'SCI',2   delta(MDRIZSKY) = 337.648   NEW MDRIZSKY = 337.648
***** skymatch ended on 2020-06-29 21:26:51.115477
TOTAL RUN TIME: 0:00:02.631066
==== Processing Step Subtract Sky finished at 21:26:51.528 (29/06/2020)
==== Processing Step  Separate Drizzle  started at  21:26:51.52 (29/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 245.92076408124387  -26.50764122005861  
CRPIX : 2108.5  2120.5  
CD1_1 CD1_2  : 4.3448008470579306e-07  1.3882091399493641e-05  
CD2_1 CD2_2  : 1.3882091399493641e-05  -4.3448008470579306e-07  
NAXIS : 4217  4241
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10146/3/j90a03frq_single_sci.fits
-Ge

                   Step          Elapsed time
   --------------------          --------------------

         Initialization          1.6042 sec.
            Static Mask          1.6285 sec.
           Subtract Sky          3.4882 sec.
       Separate Drizzle          3.7229 sec.
          Create Median          6.9921 sec.
                   Blot          10.9224 sec.
                Driz_CR          3.6971 sec.
          Final Drizzle          19.2650 sec.
   ====================          ====================
                  Total          51.3204 sec.

Trailer file written to:  astrodrizzle.log
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13364/95
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 21:27:36.362 (29/06/2020)

==== Processing Step  Initialization  started at  21:27:36.36 (29/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 210.57024618117748  54.35422173136622  
CRPI

==== Processing Step Driz_CR finished at 21:27:56.639 (29/06/2020)
==== Processing Step  Final Drizzle  started at  21:27:56.647 (29/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 210.57024618117748  54.35422173136622  
CRPIX : 2120.0  2149.0  
CD1_1 CD1_2  : -1.386469439060176e-05  8.194412871690775e-07  
CD2_1 CD2_2  : 8.194412871690775e-07  1.386469439060176e-05  
NAXIS : 4240  4298
-Generating simple FITS output: final_drc_sci.fits
Writing out image to disk: final_drc_sci.fits
Writing out image to disk: final_drc_wht.fits
Writing out image to disk: final_drc_ctx.fits
==== Processing Step Final Drizzle finished at 21:28:07.000 (29/06/2020)

AstroDrizzle Version 3.1.6 is finished processing at 21:28:07.002 (29/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          1.0450 sec.
            Static Mask  

Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10190/28/j90o28g8q_single_wht.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10190/28/j90o28fuq_single_wht.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f606w/10190/28/j90o28fjq_single_wht.fits
==== Processing Step Separate Drizzle finished at 21:28:17.894 (29/06/2020)
==== Processing Step  Create Median  started at  21:28:17.8 (29/06/2020)

reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10190/28/j90o28fjq_flc.fits' is 190.6814422607422
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10190/28/j90o28foq_flc.fits' is 188.8038330078125
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10190/28/j90o28fuq_flc.fits' is 189.65983581542972
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10190/28/j9


NOTE: Computed sky values WILL NOT be subtracted from image data ('subtractsky'=False).
'MDRIZSKY' header keyword will represent sky value *computed* from data.

-----  User specified keywords:  -----
       Sky Value Keyword:  'MDRIZSKY'
       Data Units Keyword: 'BUNIT'


-----  Input file list:  -----

   **  Input image: 'j90o13fsq_flc.fits'
       EXT: 'SCI',1;	MASK: j90o13fsq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: j90o13fsq_skymatch_mask_sci2.fits[0]

   **  Input image: 'j90o13fvq_flc.fits'
       EXT: 'SCI',1;	MASK: j90o13fvq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: j90o13fvq_skymatch_mask_sci2.fits[0]

-----  Sky statistics parameters:  -----
       statistics function: 'median'
       lower = None
       upper = None
       nclip = 5
       lsigma = 4.0
       usigma = 4.0
       binwidth = 0.1

-----  Data->Brightness conversion parameters for input files:  -----

   *   Image: j90o13fsq_flc.fits
       EXT = 'SCI',1
             Data units type

/Users/jameskwon/Documents/Research/Data/deepcr/f606w/11586/5

#### Redownloaded those for which astrodrizzle raised errors and ran astrodrizzle all again. (success)

In [26]:
astrodrizzle.AstroDrizzle('/Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/*flc.fits')

Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 08:56:19.713 (30/06/2020)

==== Processing Step  Initialization  started at  08:56:19.715 (30/06/2020)

##############################################################################
#                                                                            #
# “minmed” is highly recommended for three images,                           #
#  and is good for four to six images,                                       #
#  but should be avoided for ten or more images.                             #
#                                                                            #
##############################################################################
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 210.87794310971827  54.3520900993869  
CRPIX : 2106.5  2122.5  
CD1_1 CD1_2  : 1.3881191013961438e-05  4.623533300599064e-07  
CD2_1 CD2_2  : 4.623533300599064e

-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02dpq_single_sci.fits
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02d3q_single_sci.fits
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02dnq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02crq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02coq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02d0q_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02dpq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jco

Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02dpq_sci2_blt.fits
==== Processing Step Blot finished at 08:57:05.691 (30/06/2020)
==== Processing Step  Driz_CR  started at  08:57:05.693 (30/06/2020)

Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02coq_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02d0q_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02crq_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02d3q_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02dpq_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/err/mastDownload/HST/jco/jcoy02dnq_sci1_crmask.fits
Creating output: /Users/jameskwon/Docume

In [12]:
for prop_id in os.listdir(f435_dir):
    sub_dir = os.path.join(f435_dir,prop_id)
    if os.path.isdir(sub_dir):
        if prop_id != 'mastDownload':
            for visit_num in os.listdir(sub_dir):
                if visit_num != '.DS_Store':
                    subsub_dir = os.path.join(sub_dir,visit_num)
                    print(subsub_dir)
                    drizzle_path = subsub_dir + '/*flc.fits'
                    try:
                        astrodrizzle.AstroDrizzle(drizzle_path)
                    except:
                        pass
                    remove_extra(subsub_dir)

/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9490/a3
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 21:30:31.548 (29/06/2020)

==== Processing Step  Initialization  started at  21:30:31.550 (29/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 210.7512794192842  54.391643231480856  
CRPIX : 2107.0  2119.0  
CD1_1 CD1_2  : -1.2779112672501868e-05  -5.44017590445414e-06  
CD2_1 CD2_2  : -5.44017590445414e-06  1.2779112672501868e-05  
NAXIS : 4214  4238
********************************************************************************
*
*  Estimated memory usage:  up to 1073 Mb.
*  Output image size:       4214 X 4238 pixels. 
*  Output image file:       ~ 204 Mb. 
*  Cores available:         4
*
********************************************************************************
==== Processing Step Initialization finished at 21:30:32.410 (29/06/2020)
==== Processing Step  Static Mask  star

==== Processing Step Final Drizzle finished at 21:31:04.953 (29/06/2020)

AstroDrizzle Version 3.1.6 is finished processing at 21:31:04.955 (29/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          0.8592 sec.
            Static Mask          0.9069 sec.
           Subtract Sky          2.2505 sec.
       Separate Drizzle          3.0195 sec.
          Create Median          4.2950 sec.
                   Blot          8.0138 sec.
                Driz_CR          3.4880 sec.
          Final Drizzle          10.5485 sec.
   ====================          ====================
                  Total          33.3815 sec.

Trailer file written to:  astrodrizzle.log
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/7
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 21:31:05.067 (2

Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/7/j9es07smq_sci2_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/7/j9es07skq_sci2_crmask.fits
==== Processing Step Driz_CR finished at 21:31:24.283 (29/06/2020)
==== Processing Step  Final Drizzle  started at  21:31:24.291 (29/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 74.7665419662444  -65.99868019807921  
CRPIX : 2111.0  2148.5  
CD1_1 CD1_2  : -1.1013595519102599e-05  8.461793445216233e-06  
CD2_1 CD2_2  : 8.461793445216233e-06  1.1013595519102599e-05  
NAXIS : 4222  4297
-Generating simple FITS output: final_drc_sci.fits
Writing out image to disk: final_drc_sci.fits
Writing out image to disk: final_drc_wht.fits
Writing out image to disk: final_drc_ctx.fits
==== Processing Step Final Drizzle finished at 21:31:34.757 (29/06/2020)

AstroDrizzle Version 3.1.6 is finished processing at 21:31:34.759 (29/06/2020).



   ---------

    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/2/j9es02kvq_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/2/j9es02kvq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/2/j9es02kvq_sci2_blt.fits
==== Processing Step Blot finished at 21:31:51.25 (29/06/2020)
==== Processing Step  Driz_CR  started at  21:31:51.255 (29/06/2020)

Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/2/j9es02ktq_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/2/j9es02kvq_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/2/j9es02ktq_sci2_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/2/j9es02kvq_sci2_crmask.fits
==== Processing Step Driz_CR finished 

    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3kyq_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3kyq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3kyq_sci2_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3l1q_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3l1q_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3l1q_sci1_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3/jbfla3l1q_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generati

...

==== Processing Step Subtract Sky finished at 21:41:05.094 (29/06/2020)
==== Processing Step  Separate Drizzle  started at  21:41:05.09 (29/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 201.61972360030688  -47.533165014319316  
CRPIX : 2113.5  2268.5  
CD1_1 CD1_2  : -1.434572899389081e-06  1.3814602244155985e-05  
CD2_1 CD2_2  : 1.3814602244155985e-05  1.434572899389081e-06  
NAXIS : 4227  4537
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/3/j6lp03xlq_single_sci.fits
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/3/j6lp03y6q_single_sci.fits
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/3/j6lp03ysq_single_sci.fits
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/3/j6lp03xzq_single_sci.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/944

                  Total          58.7167 sec.

Trailer file written to:  astrodrizzle.log
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 21:41:56.518 (29/06/2020)

==== Processing Step  Initialization  started at  21:41:56.520 (29/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 201.6940184895712  -47.47195335736316  
CRPIX : 2114.5  2268.5  
CD1_1 CD1_2  : -1.4477934730484912e-06  1.3813222963063307e-05  
CD2_1 CD2_2  : 1.3813222963063307e-05  1.4477934730484912e-06  
NAXIS : 4229  4537
********************************************************************************
*
*  Estimated memory usage:  up to 2268 Mb.
*  Output image size:       4229 X 4537 pixels. 
*  Output image file:       ~ 219 Mb. 
*  Cores available:         8
*
********************************************************************************
==== Processing Step I

-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05vuq_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05vuq_sci1_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05vuq_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05vuq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05vuq_sci2_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05w3q_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5/j6lp05w3q_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435

       EXT = 'SCI',2
             Data units type: COUNTS
             EXPTIME: 1225.0 [s]
             Conversion factor (data->brightness):  0.32653061224489793

   *   Image: jc8mb1njs_flc.fits
       EXT = 'SCI',1
             Data units type: COUNTS
             EXPTIME: 1285.0 [s]
             Conversion factor (data->brightness):  0.31128404669260695
       EXT = 'SCI',2
             Data units type: COUNTS
             EXPTIME: 1285.0 [s]
             Conversion factor (data->brightness):  0.31128404669260695

   *   Image: jc8mb1nqq_flc.fits
       EXT = 'SCI',1
             Data units type: COUNTS
             EXPTIME: 1285.0 [s]
             Conversion factor (data->brightness):  0.31128404669260695
       EXT = 'SCI',2
             Data units type: COUNTS
             EXPTIME: 1285.0 [s]
             Conversion factor (data->brightness):  0.31128404669260695

   *   Image: jc8mb1nxq_flc.fits
       EXT = 'SCI',1
             Data units type: COUNTS
             EXPTIME: 128

Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nxq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nxq_sci2_blt.fits
==== Processing Step Blot finished at 21:43:21.932 (29/06/2020)
==== Processing Step  Driz_CR  started at  21:43:21.933 (29/06/2020)

Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nds_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1njs_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nqq_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nxq_sci1_crmask.fits
Creating output: /Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1/jc8mb1nds_sci2_crmask.fits
Creating output: /Users/jameskwon/Documents/Res

/Users/jameskwon/Documents/Research/Data/deepcr/f435w/13737/2

In [14]:
for prop_id in os.listdir(f814_dir):
    sub_dir = os.path.join(f814_dir,prop_id)
    if os.path.isdir(sub_dir):
        if prop_id != 'mastDownload':
            for visit_num in os.listdir(sub_dir):
                if visit_num != '.DS_Store':
                    subsub_dir = os.path.join(sub_dir,visit_num)
                    print(subsub_dir)
                    drizzle_path = subsub_dir + '/*flc.fits'
                    try:
                        astrodrizzle.AstroDrizzle(drizzle_path)
                    except:
                        pass
                    remove_extra(subsub_dir)

/Users/jameskwon/Documents/Research/Data/deepcr/f814w/9490/a2
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 21:46:03.115 (29/06/2020)

==== Processing Step  Initialization  started at  21:46:03.118 (29/06/2020)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 210.7884460492168  54.287743557165825  
CRPIX : 2107.0  2118.5  
CD1_1 CD1_2  : -1.3233067454112783e-05  -4.217482699763248e-06  
CD2_1 CD2_2  : -4.217482699763248e-06  1.3233067454112783e-05  
NAXIS : 4214  4237
********************************************************************************
*
*  Estimated memory usage:  up to 1073 Mb.
*  Output image size:       4214 X 4237 pixels. 
*  Output image file:       ~ 204 Mb. 
*  Cores available:         4
*
********************************************************************************
==== Processing Step Initialization finished at 21:46:03.930 (29/06/2020)
==== Processing Step  Static Mask  st

==== Processing Step Final Drizzle finished at 21:46:34.435 (29/06/2020)

AstroDrizzle Version 3.1.6 is finished processing at 21:46:34.437 (29/06/2020).



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          0.8124 sec.
            Static Mask          0.8979 sec.
           Subtract Sky          2.0338 sec.
       Separate Drizzle          3.0801 sec.
          Create Median          4.5359 sec.
                   Blot          5.6385 sec.
                Driz_CR          3.0720 sec.
          Final Drizzle          11.2294 sec.
   ====================          ====================
                  Total          31.2999 sec.

Trailer file written to:  astrodrizzle.log
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/16
Setting up logfile :  astrodrizzle.log
AstroDrizzle Version 3.1.6 (2020-02-19 16:16:30 -0500) started at: 21:46:34.538 (

reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/16/jbf316hkq_flc.fits' is 153.17330932617188
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/16/jbf316hnq_flc.fits' is 308.5418395996094
reference sky value for image '/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/16/jbf316hrq_flc.fits' is 204.4373779296875
Saving output median image to: 'final_med.fits'
==== Processing Step Create Median finished at 21:46:53.689 (29/06/2020)
==== Processing Step  Blot  started at  21:46:53.69 (29/06/2020)

    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/16/jbf316hhq_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/16/jbf316hhq_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/16/jbf316hhq_sci1_blt.fits

##############################################################################
#                                                                            #
# ERROR:                                                                     #
# AstroDrizzle Version 3.1.6 encountered a problem!  Processing terminated   #
# at 21:47:29.746 (29/06/2020).                                              #
#                                                                            #
##############################################################################



   --------------------          --------------------
                   Step          Elapsed time
   --------------------          --------------------

         Initialization          0.0000 sec.
   ====================          ====================
                  Total          0.0000 sec.

Trailer file written to:  astrodrizzle.log


...

-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775/52/j9l952nnq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775/52/j9l952nnq_sci2_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775/52/j9l952npq_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775/52/j9l952npq_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775/52/j9l952npq_sci1_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775/52/j9l952npq_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775/52/j9l952npq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Da


   **  Input image: 'jbqj01j0q_flc.fits'
       EXT: 'SCI',1;	MASK: jbqj01j0q_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jbqj01j0q_skymatch_mask_sci2.fits[0]

   **  Input image: 'jbqj01jhq_flc.fits'
       EXT: 'SCI',1;	MASK: jbqj01jhq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jbqj01jhq_skymatch_mask_sci2.fits[0]

   **  Input image: 'jbqj01jmq_flc.fits'
       EXT: 'SCI',1;	MASK: jbqj01jmq_skymatch_mask_sci1.fits[0]
       EXT: 'SCI',2;	MASK: jbqj01jmq_skymatch_mask_sci2.fits[0]

-----  Sky statistics parameters:  -----
       statistics function: 'median'
       lower = None
       upper = None
       nclip = 5
       lsigma = 4.0
       usigma = 4.0
       binwidth = 0.1

-----  Data->Brightness conversion parameters for input files:  -----

   *   Image: jbqj01icq_flc.fits
       EXT = 'SCI',1
             Data units type: COUNTS
             EXPTIME: 860.0 [s]
             Conversion factor (data->brightness):  0.4651162790697674
       EXT = 'SCI',2
     

Saving output median image to: 'final_med.fits'
==== Processing Step Create Median finished at 21:57:23.555 (29/06/2020)
==== Processing Step  Blot  started at  21:57:23.556 (29/06/2020)

    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/1/jbqj01icq_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/1/jbqj01icq_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/1/jbqj01icq_sci1_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/1/jbqj01icq_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/1/jbqj01icq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/1/jbqj01icq_sci2_blt.fits
  

CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 245.9587127053217  -26.598137457695845  
CRPIX : 2112.5  2125.5  
CD1_1 CD1_2  : -6.083613212263225e-06  -1.2485627130884439e-05  
CD2_1 CD2_2  : -1.2485627130884439e-05  6.083613212263225e-06  
NAXIS : 4225  4251
********************************************************************************
*
*  Estimated memory usage:  up to 2156 Mb.
*  Output image size:       4225 X 4251 pixels. 
*  Output image file:       ~ 205 Mb. 
*  Cores available:         8
*
********************************************************************************
==== Processing Step Initialization finished at 21:58:17.363 (29/06/2020)
==== Processing Step  Static Mask  started at  21:58:17.365 (29/06/2020)

==== Processing Step Static Mask finished at 21:58:18.957 (29/06/2020)
==== Processing Step  Subtract Sky  started at  21:58:18.958 (29/06/2020)

***** skymatch started on 2020-06-29 21:58:19.397426
      Version 1.0.5

'skymatch' task will apply computed sky difference

Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/2/jbqj02ggq_sci2_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/2/jbqj02ggq_sci2_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/2/jbqj02gjq_flc.fits[sci,1]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/2/jbqj02gjq_sci1_blt.fits
Writing out image to disk: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/2/jbqj02gjq_sci1_blt.fits
    Blot: creating blotted image:  /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/2/jbqj02gjq_flc.fits[sci,2]
Using default C-based coordinate transformation...
-Generating simple FITS output: /Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/2/jbqj02gjq_sci2_blt.fits
Writing out image to 

/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/1

## Bad Pixel Masks and Saturation Masks - Getting Bad pixel masks and saturation masks

F606W (test: 13018-1, 13018-2, 14343-1, 13375-2)

In [32]:
training_data = {}
test_data = {}

In [33]:
for prop_id in os.listdir(f606_dir):
    if prop_id == 'mastDownload':
        continue
    sub_dir = os.path.join(f606_dir,prop_id)
    if os.path.isdir(sub_dir):
        n_vis = !ls $sub_dir
        for vis in n_vis:
            if (prop_id == '13018' and vis == '1') or (prop_id == '13018' and vis == '2') or (prop_id == '14343' and vis == '1') or (prop_id == '13375' and vis == '2'):
                test = True
            else:
                test = False
            subsub_dir = os.path.join(sub_dir,vis)
            print(subsub_dir)
            fits_files = !ls $subsub_dir
            for file in fits_files:
                if '_crmask.fits' not in file:
                    fits_dir = os.path.join(subsub_dir, file)
                    with fits.open(fits_dir) as hdul:
                        dq = hdul[3].data
                        img = hdul[1].data
                        sat = img > 70000
                        dq = (dq & 256) & (dq & 2048) == 0

                        img2 = hdul[4].data
                        sat2 = img2 > 70000
                        dq2 = hdul[6].data
                        dq2 = (dq2 & 256) & (dq2 & 2048) == 0

                    f_crmask = fits_dir.replace('_flc.fits','_sci1_crmask.fits')
                    f_crmask2 = fits_dir.replace('_flc.fits','_sci2_crmask.fits')
                    crmask = fits.getdata(f_crmask)
                    crmask2 = fits.getdata(f_crmask2)

                    ign1 = dq | sat
                    ign2 = dq2 | sat2

                    if test:
                        test_data[f'{file}_1'] = (img, crmask, ign1)
                        test_data[f'{file}_2'] = (img2, crmask2, ign2)
                    else:
                        training_data[f'{file}_1'] = (img, crmask, ign1)
                        training_data[f'{file}_2'] = (img2, crmask2, ign2)

                    del dq, img, sat, crmask, dq2, img2, sat2, crmask2, ign1, ign2

/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/5
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10260/7
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10407/3
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/11586/5
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14235/18
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13375/4
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13375/7
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13057/1
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/14343/1
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10146/3
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10146/4
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/13364/95
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10190/13
/Users/jameskwon/Documents/Research/Data/deepcr/f606w/10190/28


F435W (test: 12103-a3, 9442-5, 13498-b1, 10631-1)

In [34]:
for prop_id in os.listdir(f435_dir):
    if prop_id == 'mastDownload':
        continue
    sub_dir = os.path.join(f435_dir,prop_id)
    if os.path.isdir(sub_dir):
        n_vis = !ls $sub_dir
        for vis in n_vis:
            if (prop_id == '12103' and vis == 'a3') or (prop_id == '9442' and vis == '5') or (prop_id == '13498' and vis == 'b1') or (prop_id == '10631' and vis == '1'):
                test = True
            else:
                test = False
            subsub_dir = os.path.join(sub_dir,vis)
            print(subsub_dir)
            fits_files = !ls $subsub_dir
            for file in fits_files:
                if '_crmask.fits' not in file:
                    fits_dir = os.path.join(subsub_dir, file)
                    with fits.open(fits_dir) as hdul:
                        dq = hdul[3].data
                        img = hdul[1].data
                        sat = img > 70000
                        dq = (dq & 256) & (dq & 2048) == 0

                        img2 = hdul[4].data
                        sat2 = img2 > 70000
                        dq2 = hdul[6].data
                        dq2 = (dq2 & 256) & (dq2 & 2048) == 0

                    f_crmask = fits_dir.replace('_flc.fits','_sci1_crmask.fits')
                    f_crmask2 = fits_dir.replace('_flc.fits','_sci2_crmask.fits')
                    crmask = fits.getdata(f_crmask)
                    crmask2 = fits.getdata(f_crmask2)

                    ign1 = dq | sat
                    ign2 = dq2 | sat2

                    if test:
                        test_data[f'{file}_1'] = (img, crmask, ign1)
                        test_data[f'{file}_2'] = (img2, crmask2, ign2)
                    else:
                        training_data[f'{file}_1'] = (img, crmask, ign1)
                        training_data[f'{file}_2'] = (img2, crmask2, ign2)

                    del dq, img, sat, crmask, dq2, img2, sat2, crmask2, ign1, ign2

/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9490/a3
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/2
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10595/7
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/12103/a3
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/1
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/3
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10631/4
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/13737/2
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/13737/3
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760/2
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/10760/4
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/1
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/3
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/9442/5
/Users/jameskwon/Documents/Research/Data/deepcr/f435w/13498/b1


F814W (test: 13737-2, 10092-1, 15336-57, 15336-10)

In [35]:
for prop_id in os.listdir(f814_dir):
    if prop_id == 'mastDownload':
        continue
    sub_dir = os.path.join(f814_dir,prop_id)
    if os.path.isdir(sub_dir):
        n_vis = !ls $sub_dir
        for vis in n_vis:
            if (prop_id == '13737' and vis == '2') or (prop_id == '10092' and vis == '1') or (prop_id == '15336' and vis == '57') or (prop_id == '15336' and vis == '10'):
                test = True
            else:
                test = False
            subsub_dir = os.path.join(sub_dir,vis)
            print(subsub_dir)
            fits_files = !ls $subsub_dir
            for file in fits_files:
                if '_crmask.fits' not in file:
                    fits_dir = os.path.join(subsub_dir, file)
                    with fits.open(fits_dir) as hdul:
                        dq = hdul[3].data
                        img = hdul[1].data
                        sat = img > 70000
                        dq = (dq & 256) & (dq & 2048) == 0

                        img2 = hdul[4].data
                        sat2 = img2 > 70000
                        dq2 = hdul[6].data
                        dq2 = (dq2 & 256) & (dq2 & 2048) == 0

                    f_crmask = fits_dir.replace('_flc.fits','_sci1_crmask.fits')
                    f_crmask2 = fits_dir.replace('_flc.fits','_sci2_crmask.fits')
                    crmask = fits.getdata(f_crmask)
                    crmask2 = fits.getdata(f_crmask2)

                    ign1 = dq | sat
                    ign2 = dq2 | sat2

                    if test:
                        test_data[f'{file}_1'] = (img, crmask, ign1)
                        test_data[f'{file}_2'] = (img2, crmask2, ign2)
                    else:
                        training_data[f'{file}_1'] = (img, crmask, ign1)
                        training_data[f'{file}_2'] = (img2, crmask2, ign2)

                        del dq, img, sat, crmask, dq2, img2, sat2, crmask2, ign1, ign2

/Users/jameskwon/Documents/Research/Data/deepcr/f814w/9490/a2
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/1
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/12
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12058/16
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/13691/11
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/9405/4b
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/9405/6d
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10092/1
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/9450/14
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/9450/16
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775/29
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775/52
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/10775/ad
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/1
/Users/jameskwon/Documents/Research/Data/deepcr/f814w/12602/2


In [227]:
np.save('training_data.npy',training_data)
np.save('test_data.npy',test_data)

## Segmentation

In [36]:
length = 256

training_segmented_data = {}
for i,pair in enumerate(training_data.items()):
    print(f'{i+1}/{len(training_data)}', end = '\r')
    f_name, val = pair
    img, mask, ignore = val
    y, x = img.shape
    y /= length
    x /= length
    idx = 0
    for yy in range(int(y)):
        for xx in range(int(x)):
            sub_img = img[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            sub_mask = mask[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            sub_ignore = ignore[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            training_segmented_data[f'{f_name}_{idx}'] = sub_img,sub_mask,sub_ignore
            idx += 1
np.save('training_segmented_data.npy',training_segmented_data)

In [37]:
length = 256

test_segmented_data = {}
for i,pair in enumerate(test_data.items()):
    print(f'{i+1}/{len(test_data)}', end = '\r')
    f_name, val = pair
    img, mask, ignore = val
    y, x = img.shape
    y /= length
    x /= length
    idx = 0
    for yy in range(int(y)):
        for xx in range(int(x)):
            sub_img = img[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            sub_mask = mask[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            sub_ignore = ignore[length*(yy):length*(yy+1),length*(xx):length*(xx+1)]
            test_segmented_data[f'{f_name}_{idx}'] = sub_img,sub_mask,sub_ignore
            idx += 1
np.save('test_segmented_data.npy',test_segmented_data)